In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
base_url= 'https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1='
id_list= ['100', '101']
news_url_list= []

In [3]:
#개별 뉴스 url 수집 및 section ID별로 id를 label로 활용하기 위해 tuple형태로 저장
for id_ in id_list:
    url= base_url+ id_
    res= requests.get(url)
    soup= BeautifulSoup(res.content, 'html.parser')
    body= soup.body.find_all('ul', class_= 'cluster_list')[0]
    for idx in range(4):
        news_url_list.append((body.find_all('li')[idx].a['href'], id_))

In [4]:
#뉴스 컨텐츠(본문) 수집 및 section ID의 끝자리를 label로 활용
news_list= []
for url, label in news_url_list:
    res= requests.get(url)
    soup= BeautifulSoup(res.content, 'html.parser')
    content= soup.body.find_all('div', class_= '_article_body_contents')[0]
    text= content.text
    news_list.append((text, label[-1]))

In [5]:
#label list생성    
real_label= [label for _, label in news_list]

In [6]:
#형태소 분석
from konlpy.tag import Okt
pos_tagger = Okt()
stopword = ['것','수','저']
total_news = []
for news, label in news_list:
    pos_news = ['/'.join(t[:-1]) for t in pos_tagger.pos(news) if ((t[1]=='Noun') & (t[0] not in stopword))]
    total_news.append(' '.join(pos_news))


In [7]:
#bow방식으로 정형화
from sklearn.feature_extraction.text import CountVectorizer
kor_vectorizer = CountVectorizer(min_df=1)
kor_bow = kor_vectorizer.fit_transform(total_news)

In [8]:
#TF-IDF로 vectorize
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer() 
tfidf = transformer.fit_transform(kor_bow.toarray())

In [9]:
#TF-IDF vector를 활용하여 KMeans로 클러스터링 
#Number of components는 section 수 할당
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2, n_init=50, verbose=1)
kmeans.fit(tfidf)

Initialization complete
Iteration  0, inertia 5.810
Iteration  1, inertia 2.497
Converged at iteration 1: center shift 0.000000e+00 within tolerance 1.137879e-07
Initialization complete
Iteration  0, inertia 4.512
Iteration  1, inertia 2.497
Converged at iteration 1: center shift 0.000000e+00 within tolerance 1.137879e-07
Initialization complete
Iteration  0, inertia 4.832
Iteration  1, inertia 2.497
Converged at iteration 1: center shift 0.000000e+00 within tolerance 1.137879e-07
Initialization complete
Iteration  0, inertia 8.974
Iteration  1, inertia 4.380
Converged at iteration 1: center shift 0.000000e+00 within tolerance 1.137879e-07
Initialization complete
Iteration  0, inertia 4.957
Iteration  1, inertia 2.497
Converged at iteration 1: center shift 0.000000e+00 within tolerance 1.137879e-07
Initialization complete
Iteration  0, inertia 6.241
Iteration  1, inertia 2.497
Converged at iteration 1: center shift 0.000000e+00 within tolerance 1.137879e-07
Initialization complete
Iter

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=50, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=1)

In [10]:
#Kmeans 라벨과 True label 비교
print(kmeans.labels_)
print(real_label)

[1 1 1 1 0 0 0 0]
['0', '0', '0', '0', '1', '1', '1', '1']
